# Hyper Paramter Tuning and efficiency Improvement Limit
### [Vedantdave117.com](https://vedantdave117.com/) || *vedantdave77@gmail.com*

In [1]:
import numpy as np
import pandas as pd

In [2]:
admission = pd.read_csv("Data/binary.csv")

In [3]:
admission.describe()

,admit,gre,gpa,rank
count,400.000000,400.000000,400.000000,400.00000
mean,0.317500,587.700000,3.389900,2.48500
std,0.466087,115.516536,0.380567,0.94446
min,0.000000,220.000000,2.260000,1.00000
25%,0.000000,520.000000,3.130000,2.00000
50%,0.000000,580.000000,3.395000,2.00000
75%,1.000000,660.000000,3.670000,3.00000
max,1.000000,800.000000,4.000000,4.00000


In [4]:
admission.shape

(400, 4)

In [5]:
admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    int64  
 1   gre     400 non-null    int64  
 2   gpa     400 non-null    float64
 3   rank    400 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 12.6 KB


### Data-Cleaning

In [6]:
# let's make dummy variable
admission_dummy = pd.get_dummies(admission['rank'],prefix = 'rank')

In [7]:
data = pd.concat([admission,admission_dummy],axis = 'columns')
data.head()

,admit,gre,gpa,rank,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,3,0,0,1,0
1,1,660,3.67,3,0,0,1,0
2,1,800,4.00,1,1,0,0,0
3,1,640,3.19,4,0,0,0,1
4,0,520,2.93,4,0,0,0,1


In [8]:
# Standardize features
for features in ['gre', 'gpa']:
    mean,std = data[features].mean(),data[features].std()
    data.loc[:,features] = (data[features]-mean)/std

### Split Data for Train and Test Dataset

In [9]:
# Split data for testing
np.random.seed(42)
sample = np.random.choice(data.index,size = int(len(data)*0.9),replace = False)
train_data, test_data = data.iloc[sample],data.drop(sample)

In [10]:
# split feature and target
features, target = train_data.drop('admit' ,axis = 'columns'), train_data['admit']
features_test ,target_test = test_data.drop('admit',axis = 'columns') ,test_data['admit']

### Define Neural_Network Model

In [11]:
# Define sigmoid activation function
def sigmoid(x):
    z = 1/(1+np.exp(-x))
    return z
def sigmoid_prime(z):
    z_prime = z * (1-z)
    return z_prime

In [12]:
# Hyper Parameters
epochs = 1000
learning_rate = 0.5

In [13]:
np.random.seed(42)
n_records, n_features = features.shape
weights = np.random.normal(scale =1 / n_features**0.5, size = n_features)

In [14]:
# looping NN and initialize weights
last_loss = None
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x,y in zip(features.values, target):
        output = sigmoid(np.dot(x,weights))
        error = y - output
        error_term = error * sigmoid_prime(output)
        del_w += error_term * x
        
    # update weight    
    weights += learning_rate * del_w/n_records
    
    # printing mean square error
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - target) ** 2)
        
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing--> Please, tuning the parameters")
        else:
            print("Train loss: ", loss)
        last_loss = loss       

Train loss:  0.3104437204888064
Train loss:  0.19815819746046254
Train loss:  0.1974337114665964
Train loss:  0.19718009683377652
Train loss:  0.19707143819100365
Train loss:  0.19701953816340484
Train loss:  0.19699303527762774
Train loss:  0.19697896255342992
Train loss:  0.19697131547543478
Train loss:  0.19696709973485202


In [15]:
# Calculate accuracy on test data
test_out = sigmoid(np.dot(features_test, weights))
predictions = test_out > 0.5
accuracy = np.mean(predictions == target_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.725


###  CONCLUSION:
### I tried several time by chaning the learning rate between rage 0.001 to 0.999, but over 0.5 learning_rate ---> overfit the model. Secondly, increase of epochs also reacts in term of high time and cost consuption. Changing the ratio of train and test data worsen the result because of (1600,4) dataset size. 

### So, its better to use multi perceptron algorithm. We may use regularization and optimizer algorithm, but will see them letter.